# Model and Feature Selection

### Load librarires

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

### Load data

In [2]:
path = os.getcwd()
path = path.replace('modeling', 'resources')
files = os.listdir(path)
for file in files:
    if len(file.split('.csv'))>1:
        csv_path = path+'/'+file
data = pd.read_csv(csv_path)

In [3]:
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


#### Apply cleaning and processing done in data preparation

In [4]:
data['SeniorCitizen']=data['SeniorCitizen'].map({0:'No', 1:'Yes'})
data['TotalCharges']=pd.to_numeric(data.TotalCharges, errors='coerce')
data.drop(data[data.TotalCharges.isna()].index, axis=0, inplace=True)
data.drop('customerID', axis=1, inplace=True)

### Data Overview

In [5]:
data.shape

(7032, 20)

In [6]:
data.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

### Strategies

#### Feature Engineering  strategy

There are a few strategies that we can try out in Feature Engineering. We will test a few base models with each of these to check the redundancy of certain features.
- The `PhoneService` might be redundant as the information is already captured in `MultipleLines`
- We will be using `One-Hot Encoding` to handle the categorical variables in the data. But some of the features (`InternetService`, `OnlineSecurity`, `OnlineBackup`, `DeviceProtection`, `TechSupport`, `StreamingTV`, `StreamingMovies`) contain the same value 'No Internet Service'. So while applying One-Hot Encoder this feature gets duplicated. We can therefore add a separate feature to show if the customer has 'No Internet Service' and set the value as 'No' in the other features.
- The `TotalCharges` feature is approximately equal to the product of `tenure` and `MonthlyCharges`. We can check how removing `TotalCharges` affects performance.
- Since `tenure` of 1 month and 72 months could potentially contain outliers, we can try by removing these as well.
- We can also try using Tenure ranges instead of the default `tenure` feature

#### Dataset Balancing Strategy

We have also seen that the dataset is imbalanced and that only around 25% of the data contains Churners. We can therefore try an over sampling and under-sampling using SMOTEEN

#### Model Selection Strategy

We can use lazy predict as a base to evaluate the features with different models. Once we are satisfied with a set of features we can select a model and tune its hyperparameters.

### Feature Engineering - 1

One-Hot Encoding with all features intact

In [34]:
from sklearn.model_selection import train_test_split

X = data.copy().drop('Churn', axis=1)
y = data['Churn'].replace({'Yes':1, 'No': 0})
cat_columns = X.select_dtypes(include='object').columns
num_columns = X.select_dtypes(include='number').columns

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [35]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

default_transformer = [('one-hot', OneHotEncoder(sparse_output=False), cat_columns), # One-Hot Encode categorical columns
                       ('scaler', MinMaxScaler(), num_columns)] #Scale Numerical columns to lie between 0 and 1
preprocessor = ColumnTransformer(default_transformer)

X_train_OH = preprocessor.fit_transform(X_train)
X_test_OH = preprocessor.transform(X_test)

X_train_OH = pd.DataFrame(X_train_OH)
X_test_OH = pd.DataFrame(X_test_OH)

In [36]:
X_train_OH.shape

(5625, 46)

In [37]:
X_train_OH.head()

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000000,0.012438,0.000081
1,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.915493,0.022886,0.154431
2,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.985915,0.717910,0.767280
3,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.000000,0.819900,0.844132
4,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000000,0.274627,0.003121
